In [1]:
#%matplotlib widget
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime,date,timedelta
from backtrader.utils import num2date
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go

In [2]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [3]:
cerebro = bt.Cerebro()
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '12/31/20'

In [4]:
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)
cerebro.addstrategy(MaCrossStrategy)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [5]:
back = cerebro.run()

0